In [2]:
import pandas as pd
from IPython.display import clear_output
def get_clone_classes(clones):
    classes=[]
    for c in range(len(clones)):
        if(clones[c].strip()=="pair start"):
            pair1=clones[c+1].strip()
            pair2=clones[c+2].strip()

            found=0
            for i in range(len(classes)):
                if(pair1 in classes[i] and pair2 in classes[i]):
                    found=1
                    break
                elif(pair1 in classes[i]):
                    classes[i].append(pair2)
                    found=1
                    break
                elif(pair2 in classes[i]):
                    classes[i].append(pair1)
                    found=1
                    break

            if(found==0):
                classes.append([pair1, pair2])    

    return classes
    

In [8]:
data_loc="H:/detected_clone_results/"
df_duplo = pd.read_csv(data_loc+"duplo_jedit_pair.csv", sep=',', header=0, encoding='utf-8')

In [27]:
data_loc="H:/detected_clone_results/"
df_duplo = pd.read_csv(data_loc+"duplo_jedit_pair.csv", sep=',', header=0, encoding='utf-8')
working_revision=3791 #starting version
classes=[]
query=""
for i in range(0, len(df_duplo), 2):
    if(working_revision!=df_duplo["version"][i]):
        clear_output(wait=True)
        print("Working for :", working_revision)    
        #print("*****Start Here****")      
        
        class_id=0
        for cls in classes:
            class_to_take=[]    
            for c in cls:
                #print(working_revision, class_id, c)
                if(c[0:5]=="trunk"):
                    class_to_take.append(c.split())
            
            if(len(class_to_take)>1):
                query="INSERT INTO `clones_duplo` (`version`, `CloneClass`, `file_name`, `startline`, `endline`) VALUES \n"
                for ct in class_to_take:
                    #print(class_id, ct)
                    query+="('"+str(working_revision)+"', '"+str(class_id)+"', '"+ct[0]+"', '"+ct[1]+"', '"+ct[2]+"'),\n"
                class_id+=1
        
        #print(query)           
        #print("######")    
        if(len(query)>1):
            with open(data_loc+"duplo_db_query/revision-"+str(working_revision)+".sql", "w") as text_file:
                text_file.write(query[0:-2]+";")
        query=""
        
        classes=[]#Re-initialize the class for next revision
        working_revision=df_duplo["version"][i]
        
        
        
    #print(df_duplo["version"][i], df_duplo["ClonePair"][i], df_duplo["file_name"][i], df_duplo["startline"][i], df_duplo["endline"][i])
    #cur_version=df_duplo["version"][i]
    pair1=df_duplo["file_name"][i]+", "+str(df_duplo["startline"][i])+", "+str(df_duplo["endline"][i])
    pair2=df_duplo["file_name"][i+1]+", "+str(df_duplo["startline"][i+1])+", "+str(df_duplo["endline"][i+1])
    
    #print(df_duplo["version"][i], pair1, pair2)
    
    found=0
    for i in range(len(classes)):
        if(pair1 in classes[i] and pair2 in classes[i]):
            found=1
            break
        elif(pair1 in classes[i]):
            classes[i].append(pair2)
            found=1
            break
        elif(pair2 in classes[i]):
            classes[i].append(pair1)
            found=1
            break

    if(found==0):
        classes.append([pair1, pair2])    
    
    
    
    
#     if(df_duplo["version"][i]>3793):
#         break
        
        

Working for : 3991


In [ ]:
for rev in range(3791, 4002):
    clear_output(wait=True)
    print("Working for :", rev)    
    clones=open(data_loc+"version-"+str(rev)+".txt","r").read().strip().split('\n') 
    clone_classes=get_clone_classes(clones)
    
    #rev=3900
    query="INSERT INTO `clones_ccfinder` (`version`, `CloneClass`, `file_name`, `startline`, `endline`) VALUES \n"
    class_id=0
    for cls in clone_classes:
        class_to_take=[]    
        for c in cls:
            c=c.replace("\\", "/").replace("H:/jedit/repository/version-"+str(rev)+"/", "")
            if(c[0:5]=="trunk"):
                class_to_take.append(c.split())

        if(len(class_to_take)>1):
            for ct in class_to_take:
                #print(class_id, ct)
                query+="('"+str(rev)+"', '"+str(class_id)+"', '"+ct[0]+"', '"+ct[1]+"', '"+ct[2]+"'),\n"
            class_id+=1

    #print(query[0:-2])
    #Now write the generated query to a SQL file.......
    with open(data_loc+"../final_db_query/revision-"+str(rev)+".sql", "w") as text_file:
            text_file.write(query[0:-2])

Working for : 3905
